# Statistical Data Management Session 8: Central Limit Theorem and Confidence Intervals (chapters 6-7 in McClave & Sincich)

## 1. Illustrating the Central Limit Theorem

1. Run the code below. A uniform distribution is defined, and 1000 samples of size ``n = 1`` are drawn. Then, the means of these size 1 samples (i.e., these values themselves in this case!) are collected in the list ``means``. Then a histogramme of these is generated. This closely resembles the probability distribution of the uniform distribution, which is to be expected.
2. Set ``n = 2`` and re-run. Now, 1000 samples of size 2 are drawn and the sample means recalculated. The histogramme tends to resemble a normal curve, the central limit theorem in action. 
3. Uncomment another distribution. Rerun the code with ``n = 2``. Because (unlike the uniform distribution) these are asymmetric, a sample size of 2 is not sufficient to see the effect of the CLT. Increase the sample size to see the effect. Note the smaller range of the histogramme categories, because the sample mean follows a normal distribution with a standard deviation divided by $\sqrt{n}$.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as sts
%matplotlib inline

distr = sts.uniform(0,2)
# alternatives, don't worry about them, we won't use them further, just here for illustration purposes!
#distr = sts.expon()
#distr = sts.gibrat()
#distr = sts.triang(0.99)

interval = np.linspace(-0.5,2.5,10000)
plt.plot(interval, distr.pdf(interval), linewidth=3)
plt.show()
plt.close()

n = 1
means = np.empty(1000)  # empty array to save sample means

for i in range(1000):
    vals = distr.rvs(size = n) # rvs = random variates => draw sample of given size at random from distribution
    means[i] = vals.mean() # save sample mean

plt.hist(means, bins='sqrt')
plt.show()
plt.close()

## 2. Central Limit Theorem *(based on ex 6.62 from the book)*

**In order to fully grasp the CLT, we will abstract away from the concrete story given in the book.**

A certain population quantity has an unknown distribution with a mean of 5.1 and a standard deviation of 6.1. Consider a random sample of size $n=150$. Let $\bar{x}$ represent the sample mean.
1. Give the expected value and standard deviation of the sampling distribution of $\bar{x}$.

$E(\bar{x})=5.1, \sigma_\bar{x} = \frac{6.1}{\sqrt{150}}$.

2. Will the sampling distribution of $\bar{x}$ be approximately normal? Explain.

Yes, by virtue of the central limit theorem and the fact that 150 > 30. $\bar{x} \sim N(5.1, \frac{6.1}{\sqrt{150}})$.

3. Find $P(\bar{x} > 5.5)$.

$= 1 - P(\bar{x} < 5.5) = 1-P(z < \frac{5.5-5.1}{6.1/\sqrt{150}})= 1 - 0.79 = 0.21$. ($0.79$ obtained by code below)

4. Find $P(4< \bar{x} <5)$.

$= P(\bar{x} <5) - P(\bar{x} < 4) = 0.42 - 0.01 = 0.41$. (similarly, refer to code below)

5. A second population has a distribution with a mean of 5.4 and a standard deviation of 0.5. Again, consider a random sample of size $n=150$. Say you have a sample with sample mean 5.5. Is it more likely that this sample was drawn from the first or the second population?

$\bar{x}_2 \sim N(5.4, \frac{0.5}{\sqrt{150}}), P(\bar{x}_2 > 5.5) = \cdots = 0.01$, so it's more likely that the sample was drawn from the first population. Note that the sample mean was closer to the expected value of the second population than that of the first, but the second had a smaller standard deviation. This can be easily spotted in the plots of the sampling distributions.

Code:

In [ ]:
mu_1 = 5.1
sigma_1 = 6.1
n = 150
mu_x_bar_1 = mu_1
sigma_x_bar_1 = sigma_1 / np.sqrt(n)

# distribution of x_bar_1
x_bar_distribution_1 = sts.norm(mu_x_bar_1 , sigma_x_bar_1)
print(1 - x_bar_distribution_1.cdf(5.5))
print(x_bar_distribution_1.cdf(5) - x_bar_distribution_1.cdf(4))


# distribution of x_bar_2
mu_2 = 5.4
sigma_2 = 0.5
mu_x_bar_2 = mu_2
sigma_x_bar_2 = sigma_2 / np.sqrt(n)
x_bar_distribution_2 = sts.norm(mu_x_bar_2 , sigma_x_bar_2)

print(1 - x_bar_distribution_2.cdf(5.5))

interval = np.linspace(4, 6.5, 10000)
part = np.linspace(5.5, 6.5, 10000)
plt.figure(figsize=(10,6))
plt.plot(interval,x_bar_distribution_1.pdf(interval), label = "sampling distribution for population 1")
plt.plot(interval,x_bar_distribution_2.pdf(interval), label = "sampling distribution for population 2")
plt.fill_between(part, x_bar_distribution_1.pdf(part))
plt.fill_between(part, x_bar_distribution_2.pdf(part))
plt.legend()
plt.show()
plt.close()

## 3. Confidence interval: proportion of smokers

In the 2018 Belgian Health Interview Survey (HIS) (https://www.sciensano.be/nl/projecten/gezondheidsenquete-0#levensstijl), 19.4% of the people interviewed answered "yes" to the question whether they smoke or not. 10700 people were interviewed. 

1. Construct a 95% confidence interval for the true proportion of smokers. Use Python where necessary. Hint: use the method ``ppf(...)`` on a standard normal distribution to find the z-value given a confidence level. ``ppf`` is the inverse of ``cdf``.

Both $pn$ and $qn$ are (way) larger than 15, so we construct the large sample confidence interval:

$z_{\alpha/2} = 1.96$ (from code below)

$\left[0.194 - 1.96\sqrt{\frac{0.194\cdot0.806}{10700}}, 0.194 + 1.96\sqrt{\frac{0.194\cdot0.806}{10700}}\right] = [0.187,0.201]$.

In [ ]:
p_hat = 0.194
q_hat = 1 - p_hat
n = 10700
alpha = 0.05
standard_normal = sts.norm(0,1)
z_alphaDiv2 = standard_normal.ppf(1 - alpha/2)
term = z_alphaDiv2 * np.sqrt(p_hat * q_hat / n)
interval = [p_hat - term, p_hat + term]
print(interval)

2. For prevention purposes, this accuracy is not strictly necessary. Say that for their next survey, the government wants an estimate up to 2% of the true proportion. How many interviews are needed, starting from the estimate of this year?

In this case, we want $0.02 = 1.96\sqrt{\frac{0.194\cdot0.806}{n}} \Leftrightarrow n = \frac{0.194\cdot0.806\cdot1.96^2}{0.02^2} = 1501.72$, so take $n=1502$.

In [ ]:
error = 0.02
n = p_hat * q_hat / (error/z_alphaDiv2)**2
print(n)

## 4. Confidence interval: birthweight

The file ``birthweights.csv`` in the ``shared`` folder contains the birthweight in kg of a sample of 42 new-born babies (https://www.sheffield.ac.uk/mash/statistics/datasets). Run the following cell of code and construct a 90% confidence interval for the true birthweight of new-born babies.

In [ ]:
df_birthweights = pd.read_csv("../../shared/birthweights.csv")
print(df_birthweights["Birthweight"])

In [ ]:
n = len(df_birthweights)
print(n) # > 30 => large sample confidence interval

x_bar = df_birthweights["Birthweight"].mean()
s = df_birthweights["Birthweight"].std()

alpha = 0.10
standard_normal = sts.norm(0,1)
z_alphaDiv2 = standard_normal.ppf(1 - alpha/2)

term = z_alphaDiv2 * s / np.sqrt(n)

interval = [x_bar - term, x_bar + term]
print(interval)
print(x_bar)
print(term)

## 5. Confidence interval: birthweight smokers

The file ``birthweight_smokers.csv`` in the ``shared`` folder contains the birthweight in kg of a sample of 22 new-born babies from mothers who smoked during pregnancy (a subset of the sample from exercise 4). 

1. Run the following cell of code and construct a 90% confidence interval for the true birthweight of new-born babies from smokers. You may assume that birthweights follow a normal distribution. Show with a QQ plot and/or histogram that this assumption is not unrealistic.

In [ ]:
df_birthweights_smokers = pd.read_csv("../../shared/birthweights_smokers.csv")
print(df_birthweights_smokers["Birthweight"])

In [ ]:
# Probability plot and histogram to show that the assumption of normality is not unrealistic:

plt.figure(figsize=(10,6))   
sts.probplot(df_birthweights["Birthweight"], plot=plt)
plt.show()
plt.close()

plt.figure(figsize=(10,6))   
plt.hist(df_birthweights["Birthweight"], bins='sqrt')
plt.show()
plt.close()

In [ ]:
n = len(df_birthweights_smokers)
print(n) # < 30 => small sample confidence interval

x_bar = df_birthweights_smokers["Birthweight"].mean()
s = df_birthweights_smokers["Birthweight"].std()

alpha = 0.10
t_distribution = sts.t(n - 1)
t_alphaDiv2 = t_distribution.ppf(1 - alpha/2)

# Plot to show that the t-distribution has slightly "heavier tails"
interval = np.linspace(-4, 4, 10000)
plt.figure(figsize=(10,6))
plt.plot(interval, standard_normal.pdf(interval), label = "Standard normal distribution")
plt.plot(interval, t_distribution.pdf(interval), label = "T-distribution with 21 degrees of freedom")
plt.legend()
plt.show()
plt.close()

term = t_alphaDiv2 * s / np.sqrt(n)
interval = [x_bar - term, x_bar + term]
print(interval)

2. What would change if you knew the population standard deviation of birthweights?

In that case you could use the large sample confidence interval!

## 6. SQL recap

The file ``birthweights.sql`` provided on Toledo contains the information used in exercises 4 and 5: a table listing baby IDs and their corresponding birthweight, and a table listing the smoker status of their mother (0 for non-smoking, 1 smoking). Import the file using MySQL Workbench and write the appropriate queries to retrieve the relevant information. Re-run your analysis (without running the cell which defined the dataframe!) to check whether you have the correct information.

In [ ]:
conn = sqlite3.connect("../../shared/birthweights.db")
df_birthweights = pd.read_sql_query("SELECT Birthweight FROM weights", conn)
print(df_birthweights)

query = """
SELECT Birthweight 
FROM weights JOIN smokerstatus 
ON weights.ID = smokerstatus.ID 
WHERE smoker = 1
"""
df_birthweights_smokers = pd.read_sql_query(query, conn)

# Alternatively:
query_2 = """
SELECT Birthweight 
FROM weights 
WHERE ID IN 
   (SELECT ID FROM smokerstatus WHERE smoker = 1)
"""
df_birthweights_smokers = pd.read_sql_query(query_2, conn)

print(df_birthweights_smokers)